In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
from tensorflow.python.ops import lookup_ops

import numpy as np

In [2]:
tf.enable_eager_execution()

In [3]:
tf.set_random_seed(42)

In [4]:
from rnn_lm import LanguageModel, create_dataset, loss_fun

In [5]:
train_file = 'simple-examples/data/ptb.train.txt'
valid_file = 'simple-examples/data/ptb.valid.txt'
vocab_file = 'simple-examples/data/vocab.txt'

In [6]:
vocab_table = lookup_ops.index_table_from_file(vocab_file)

In [7]:
#Specify model params
V = int(vocab_table.size())
cell = 'lstm'
d = 128
h = 128

BATCH_SIZE=32

In [8]:
lm = LanguageModel(V=V, cell=cell, d=d, h=h)

In [9]:
train_dataset = create_dataset(train_file, batch_size=BATCH_SIZE, vocab_table=vocab_table)

In [10]:
datum = next(iter(train_dataset))

In [11]:
loss_fun(lm, datum)

<tf.Tensor: id=1796, shape=(), dtype=float32, numpy=9.210991>

In [12]:
checkpoint_dir = 'lm'

In [13]:
opt = tf.train.AdamOptimizer(learning_rate=0.001)

In [14]:
root = tfe.Checkpoint(optimizer=opt, model=lm, optimizer_step=tf.train.get_or_create_global_step())

In [15]:
root.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [16]:
tf.train.get_or_create_global_step()

<tf.Variable 'global_step:0' shape=() dtype=int64, numpy=159>

In [17]:
loss_fun(lm, datum)

<tf.Tensor: id=3608, shape=(), dtype=float32, numpy=6.844154>

### Resume training!

In [18]:
loss_and_grads_fun = tfe.implicit_value_and_gradients(loss_fun)

In [19]:
NUM_EPOCHS = 10
STATS_STEPS = 50
for epoch_num in range(NUM_EPOCHS):
    batch_loss = []
    for step_num, datum in enumerate(train_dataset, start=1):
        loss_value, gradients = loss_and_grads_fun(lm, datum)
        batch_loss.append(loss_value)
        
        if step_num % STATS_STEPS == 0:
            print(f'Epoch: {epoch_num} Step: {step_num} Avg Loss: {np.average(np.asarray(loss_value))}')
            batch_loss = []
        opt.apply_gradients(gradients, global_step=tf.train.get_or_create_global_step())
    print(f'Epoch{epoch_num} Done!')

Epoch: 0 Step: 50 Avg Loss: 6.777772426605225
Epoch: 0 Step: 100 Avg Loss: 6.5137715339660645


KeyboardInterrupt: 